# LIBRERIAS 

* Tener en cuenta bajadas de Procedimientos, Personas Armas, Divisas , vehiculos (secuestrados, ministerio), Narcotrafico general,Objetos y pegar en la carpeta bajadas de fecha del primero del 1 del mes en informar hasta fecha del informe 

### IMPORTACIÓN DE LIBREARIAS


In [1]:
import pandas as pd
from  Parametros import * 
from  Funciones import * 
from openpyxl import load_workbook , Workbook
import os
from datetime import datetime
from openpyxl.styles import Alignment


### PARAMETROS

In [2]:
PATH_BASE = 'db/base_informada.xlsx'
PATH_PROCEDIMIENTOS = 'bajadas/bajada_general.xls'
PATH_ARMAS = 'bajadas/bajada_arma.xls'
PATH_DIVISAS = 'bajadas/bajada_divisas_secuestradas.xls'
PATH_NARCOTRAFICO = 'bajadas/bajada_general_narcotrafico.xls'
PATH_OBJETOS = 'bajadas/bajada_objetos.xls'
PATH_PERSONAS = 'bajadas/bajada_general_persona.xls'
PATH_VEHICULOS = 'bajadas/bajada_vehiculos_secuestrados.xls'
PATH_OPERACIONES = 'bajadas/PLANILLA DE CARGA DE DATOS.xlsx'
PATH_TRATA = 'bajadas/bajada_trata.xls'


### ACCEDER BASE DE DATOS

In [3]:
try:
    # Intentar leer el archivo
    df_base_informada = pd.read_excel(
        PATH_BASE,
        sheet_name="GEOG. PROCEDIMIENTO",  # Hoja como cadena, no como lista
        skiprows=2,# Saltar las dos primeras filas
        usecols=lambda col: True if col != 0 else False 
        
    )
    
    # Operaciones con el DataFrame si se lee correctamente
    print("Información del DataFrame:")
    print(df_base_informada.info())  # Información general del DataFrame
    print("\nDescripción estadística:")
    print(df_base_informada.describe())  # Estadísticas descriptivas
    print("\nValores únicos por columna:")
    print(df_base_informada.nunique())  

except FileNotFoundError:
    # Manejar el error de archivo no encontrado
    print("El archivo 'base_informada.xlsx' no fue encontrado en la ruta 'db/'. Verifica su existencia.")
    df_base_informada = pd.DataFrame()
    print(df_base_informada)

El archivo 'base_informada.xlsx' no fue encontrado en la ruta 'db/'. Verifica su existencia.
Empty DataFrame
Columns: []
Index: []


# SIGIPOL


### PROCEDIMIENTOS GENERALES




In [4]:
excel_bajada_procedimientos_generales = filtrar_procedimientos_generales(PATH_PROCEDIMIENTOS)

df_procedimientos = pd.DataFrame()
df_procedimientos['ID_OPERATIVO'] = excel_bajada_procedimientos_generales.apply(procesar_causa_judicial, axis=1)
df_procedimientos['FUERZA_INTERVINIENTE'] = "PSA"
df_procedimientos['ID_PROCEDIMIENTO'] = excel_bajada_procedimientos_generales.apply(generar_uid_sigpol, axis=1)
df_procedimientos['CAUSAJUDICIALNUMERO'] = excel_bajada_procedimientos_generales['CAUSAJUDICIALNUMERO'].copy()
df_procedimientos['UNIDAD_INTERVINIENTE'] = excel_bajada_procedimientos_generales['UOSP'].fillna(excel_bajada_procedimientos_generales['URSA'])
df_procedimientos['DESCRIPCIÓN'] = excel_bajada_procedimientos_generales.apply(procesar_descripcion, axis=1)
df_procedimientos['TIPO_INTERVENCION'] = excel_bajada_procedimientos_generales.apply(procesar_tipo_procedimiento, axis=1)
df_procedimientos['PROVINCIA'] = excel_bajada_procedimientos_generales.apply(procesar_provincia, axis=1)
df_procedimientos['DEPARTAMENTO O PARTIDO'] = excel_bajada_procedimientos_generales.apply(procesar_municipio, axis=1)
df_procedimientos['LOCALIDAD'] = "-"
df_procedimientos['DIRECCION'] = excel_bajada_procedimientos_generales.apply(procesar_direccion, axis=1)
df_procedimientos[['LATITUD', 'LONGITUD']] = excel_bajada_procedimientos_generales.apply(procesar_geog, axis=1, result_type='expand')
df_procedimientos['FECHA'] = pd.to_datetime(excel_bajada_procedimientos_generales['DENUNCIAFECHA'], errors='coerce').dt.strftime('%d/%m/%Y')
df_procedimientos['HORA'] = pd.to_datetime(excel_bajada_procedimientos_generales['DENUNCIAFECHA'], errors='coerce').dt.strftime('%H:%M')
df_procedimientos['ZONA_SEGURIDAD_FRONTERAS'] = "-"
df_procedimientos['PASO_FRONTERIZO'] = "-"
df_procedimientos['OTRAS AGENCIAS INTERVINIENTES'] = "-"
df_procedimientos['Observaciones - Detalles'] = "-"


df_procedimientos['LATITUD'] = df_procedimientos['LATITUD'].astype(str).str.replace(',', '.')
df_procedimientos['LONGITUD'] = df_procedimientos['LONGITUD'].astype(str).str.replace(',', '.')

df_procedimientos_completado = df_procedimientos[['FUERZA_INTERVINIENTE', 'ID_OPERATIVO', 'ID_PROCEDIMIENTO',
                                     'UNIDAD_INTERVINIENTE', 'DESCRIPCIÓN', 'TIPO_INTERVENCION',
                                     'PROVINCIA', 'DEPARTAMENTO O PARTIDO', 'LOCALIDAD', 'DIRECCION',
                                     'ZONA_SEGURIDAD_FRONTERAS', 'PASO_FRONTERIZO', 'LATITUD', 'LONGITUD',
                                     'FECHA', 'HORA', 'OTRAS AGENCIAS INTERVINIENTES', 'Observaciones - Detalles']]

print(df_procedimientos_completado.nunique())


Estadistica de Partes

Total de Partes: 327
Cantidad Duplicado: 0
Cantidad No diponible: 28
Cantidad de Partes final: 327
FUERZA_INTERVINIENTE               1
ID_OPERATIVO                     312
ID_PROCEDIMIENTO                 327
UNIDAD_INTERVINIENTE              33
DESCRIPCIÓN                        8
TIPO_INTERVENCION                  2
PROVINCIA                         21
DEPARTAMENTO O PARTIDO            26
LOCALIDAD                          1
DIRECCION                         21
ZONA_SEGURIDAD_FRONTERAS           1
PASO_FRONTERIZO                    1
LATITUD                           78
LONGITUD                          81
FECHA                             26
HORA                             173
OTRAS AGENCIAS INTERVINIENTES      1
Observaciones - Detalles           1
dtype: int64


### FILTRO CON LA BASE DE DATOS DE LO QUE SE FUE INFORMADO




In [5]:
if  df_base_informada.empty:
    print("No hay casos en la base de datos ")
    df_procedimientos_final = df_procedimientos_completado
else:
    df_procedimientos_final = df_procedimientos_completado[~df_procedimientos_completado['ID_PROCEDIMIENTO'].isin(set(df_base_informada['ID_PROCEDIMIENTO']))]
    
print(f"\nCasos ya informados: {len(df_base_informada)}")
print(f"Casos nuevos para informar : {len(df_procedimientos_final)}")

print(df_procedimientos_final.nunique())




No hay casos en la base de datos 

Casos ya informados: 0
Casos nuevos para informar : 327
FUERZA_INTERVINIENTE               1
ID_OPERATIVO                     312
ID_PROCEDIMIENTO                 327
UNIDAD_INTERVINIENTE              33
DESCRIPCIÓN                        8
TIPO_INTERVENCION                  2
PROVINCIA                         21
DEPARTAMENTO O PARTIDO            26
LOCALIDAD                          1
DIRECCION                         21
ZONA_SEGURIDAD_FRONTERAS           1
PASO_FRONTERIZO                    1
LATITUD                           78
LONGITUD                          81
FECHA                             26
HORA                             173
OTRAS AGENCIAS INTERVINIENTES      1
Observaciones - Detalles           1
dtype: int64


### HOJA PERSONAS  

In [6]:
# Paso 1: Cargar datos iniciales
excel_bajada_personas = pd.read_excel(PATH_PERSONAS)
print(f"Total de registros en la bajadas de personas: {len(excel_bajada_personas)}")

# Paso 2: Generar UID
excel_bajada_personas['UOSP'] = excel_bajada_personas['UOSP'].fillna(excel_bajada_personas['URSA'])
excel_bajada_personas['TIPO_CAUSA_INTERNA'] = excel_bajada_personas.apply( procesar_tipo_causa_interna, axis=1)
excel_bajada_personas['ID_PROCEDIMIENTO'] = excel_bajada_personas.apply(generar_uid_sigpol, axis=1)

# Paso 3: Coincidencia de UID entre excel_bajada_personas y df_geog_final
uid_coincidentes = excel_bajada_personas[excel_bajada_personas['ID_PROCEDIMIENTO'].isin(df_procedimientos_final['ID_PROCEDIMIENTO'])]

# Crear DataFrame de detenidos y aprendidos
df_detenidos_aprendidos = pd.DataFrame()
df_detenidos_aprendidos['ID_PROCEDIMIENTO'] = uid_coincidentes['ID_PROCEDIMIENTO']
df_detenidos_aprendidos['EDAD'] = uid_coincidentes.apply(procesar_edad, axis=1)
df_detenidos_aprendidos['SEXO'] = uid_coincidentes.apply(procesar_sexo, axis=1)
df_detenidos_aprendidos['GENERO'] = uid_coincidentes.apply(procesar_genero, axis=1)
df_detenidos_aprendidos['NACIONALIDAD'] = uid_coincidentes.apply(procesar_nacionalidad, axis=1)
df_detenidos_aprendidos['SITUACION_PROCESAL'] = uid_coincidentes.apply(procesar_situacion_judicial, axis=1)
df_detenidos_aprendidos['DELITO_IMPUTADO'] = uid_coincidentes.apply(procesar_tipo_delito, axis=1)
df_detenidos_aprendidos['JUZGADO_INTERVINIENTE'] = uid_coincidentes.apply(procesar_juzgado, axis=1)
df_detenidos_aprendidos['CARATULA_CAUSA'] = uid_coincidentes.apply(procesar_caratula, axis=1)
df_detenidos_aprendidos['NUM_CAUSA'] = uid_coincidentes['ID_PROCEDIMIENTO']


df_detenidos_aprendidos = df_detenidos_aprendidos[df_detenidos_aprendidos['SITUACION_PROCESAL'] != 'NO INFORMAR']
df_detenidos_aprendidos = df_detenidos_aprendidos[df_detenidos_aprendidos['ID_PROCEDIMIENTO'].isin(df_procedimientos_final['ID_PROCEDIMIENTO'])]


print(df_detenidos_aprendidos.nunique())

Total de registros en la bajadas de personas: 377
ID_PROCEDIMIENTO         281
EDAD                      56
SEXO                       2
GENERO                     2
NACIONALIDAD              19
SITUACION_PROCESAL         6
DELITO_IMPUTADO           23
JUZGADO_INTERVINIENTE     76
CARATULA_CAUSA            61
NUM_CAUSA                281
dtype: int64


### HOJA OTROS DELITOS

In [7]:
df_otros_delitos = df_detenidos_aprendidos[df_detenidos_aprendidos['SITUACION_PROCESAL'] == 'VICTIMA']
# Filtro los que son victima de detenidos y aprendidos
df_detenidos_aprendidos_completado = df_detenidos_aprendidos[df_detenidos_aprendidos['SITUACION_PROCESAL'] != 'VICTIMA']

# Cambiar el nombre de algunas columnas en el DataFrame de víctimas
df_otros_delitos_completado = df_otros_delitos.rename(columns={
    'DELITO_IMPUTADO': 'TIPO_OTRO_DELITO',
    'GENERO': 'GENERO_VICTIMA',
    'EDAD': 'EDAD_VICTIMA',
})
df_otros_delitos_completado["OBSERVACIONES"] ="-"

print(df_otros_delitos_completado.nunique())


ID_PROCEDIMIENTO         137
EDAD_VICTIMA              55
SEXO                       2
GENERO_VICTIMA             2
NACIONALIDAD              12
SITUACION_PROCESAL         1
TIPO_OTRO_DELITO          16
JUZGADO_INTERVINIENTE     21
CARATULA_CAUSA             7
NUM_CAUSA                137
OBSERVACIONES              1
dtype: int64


### INCAUTACIONES ARMA

In [8]:
# Paso 1: Cargar datos iniciales
excel_bajada_arma = pd.read_excel(PATH_ARMAS)
print(f"Total de registros cargados: {len(excel_bajada_arma)}")

excel_bajada_arma = excel_bajada_arma[excel_bajada_arma["TIPO_ESTADO_OBJETO"] == "SECUESTRADO"]
excel_bajada_arma['UOSP'] = excel_bajada_arma['UOSP'].fillna(excel_bajada_arma['URSA'])
excel_bajada_arma['TIPO_CAUSA_INTERNA'] = excel_bajada_arma.apply( procesar_tipo_causa_interna, axis=1)
excel_bajada_arma['ID_PROCEDIMIENTO'] = excel_bajada_arma.apply(generar_uid_sigpol, axis=1)


df_arma = pd.DataFrame()
df_arma['ID_PROCEDIMIENTO'] = excel_bajada_arma['ID_PROCEDIMIENTO']
df_arma['TIPO_INCAUTACION'] = "ARMAS"
df_arma['TIPO'] = excel_bajada_arma['TIPO_ARMA']
df_arma['SUBTIPO'] = "-"
df_arma['CANTIDAD'] = excel_bajada_arma.apply(procesar_cantidad_arma, axis=1)
df_arma['MEDIDAS'] = "UNIDADES"
df_arma['AFORO'] = "-"
df_arma['OBSERVACIONES'] = excel_bajada_arma.apply(procesar_observaciones_arma, axis=1)

df_arma_completado = df_arma[df_arma['ID_PROCEDIMIENTO'].isin(df_procedimientos_final['ID_PROCEDIMIENTO'])]


print(df_arma_completado.nunique())

Total de registros cargados: 8
ID_PROCEDIMIENTO    5
TIPO_INCAUTACION    1
TIPO                4
SUBTIPO             1
CANTIDAD            3
MEDIDAS             1
AFORO               1
OBSERVACIONES       7
dtype: int64


### INCAUTACIONES DIVISA

In [9]:
excel_bajada_divisa = pd.read_excel(PATH_DIVISAS)
print(f"Total de registros cargados: {len(excel_bajada_divisa)}")

excel_bajada_procedimientos_generales['PARTE_ANIO'] = excel_bajada_procedimientos_generales['NUMERO_PARTE'].astype(str) + "/" + excel_bajada_procedimientos_generales['ANIO_PARTE'].astype(str)
excel_bajada_divisa['PARTE_ANIO'] = excel_bajada_divisa['NUMERO_PARTE'].astype(str) + "/" + excel_bajada_divisa['ANIO_PARTE'].astype(str)

excel_bajada_divisa['UOSP'] = excel_bajada_divisa['UOSP'].fillna(excel_bajada_divisa['URSA'])

excel_bajada_divisa = pd.merge(excel_bajada_divisa, 
                               excel_bajada_procedimientos_generales[['PARTE_ANIO', 'TIPO_CAUSA_INTERNA']], 
                               on='PARTE_ANIO', 
                               how='left')

excel_bajada_divisa['ID_PROCEDIMIENTO'] = excel_bajada_divisa.apply(generar_uid_sigpol, axis=1)
excel_bajada_divisa = excel_bajada_divisa[~pd.isnull(excel_bajada_divisa['TOTAL_DIVISAS_SECUESTRADAS'])]


# Agrupar por 'UID' (y 'TIPO' si se necesita) para sumar las cantidades
cantidad_por_uid = excel_bajada_divisa.groupby(['ID_PROCEDIMIENTO', 'TIPO_DIVISA'], as_index=False)['TOTAL_DIVISAS_SECUESTRADAS'].sum()

# Paso 2: Crear df_divisa con las cantidades sumadas
df_divisa = pd.DataFrame()
df_divisa['ID_PROCEDIMIENTO'] = cantidad_por_uid['ID_PROCEDIMIENTO']
df_divisa['TIPO_INCAUTACION'] = "DIVISAS"
df_divisa['TIPO'] = cantidad_por_uid['TIPO_DIVISA']
df_divisa['SUBTIPO'] = "-"
df_divisa['CANTIDAD'] = cantidad_por_uid['TOTAL_DIVISAS_SECUESTRADAS']
df_divisa['MEDIDAS'] = "UNIDADES"
df_divisa['AFORO'] = "-"
df_divisa['OBSERVACIONES'] = "-"

# Mostrar el resultado final
df_divisa_completado = df_divisa[df_divisa['ID_PROCEDIMIENTO'].isin(df_procedimientos_final['ID_PROCEDIMIENTO'])]

print(df_divisa_completado.nunique())

Total de registros cargados: 59
ID_PROCEDIMIENTO    11
TIPO_INCAUTACION     1
TIPO                 5
SUBTIPO              1
CANTIDAD            17
MEDIDAS              1
AFORO                1
OBSERVACIONES        1
dtype: int64


### INCAUTACIONES OBJETOS

In [10]:
# Paso 1: Cargar datos iniciales
excel_bajada_objetos = pd.read_excel(PATH_OBJETOS)
print(f"Total de registros cargados: {len(excel_bajada_objetos)}")

excel_bajada_objetos = excel_bajada_objetos[excel_bajada_objetos["ESTADO"] == "SECUESTRADO"]

excel_bajada_objetos['UOSP'] = excel_bajada_objetos['UOSP'].fillna(excel_bajada_objetos['URSA'])
excel_bajada_objetos['TIPO_CAUSA_INTERNA'] = excel_bajada_objetos.apply( procesar_tipo_causa_interna, axis=1)
excel_bajada_objetos['ID_PROCEDIMIENTO'] = excel_bajada_objetos.apply(generar_uid_sigpol, axis=1)
excel_bajada_objetos = excel_bajada_objetos[excel_bajada_objetos['TIPO_OBJETO'].isin(TIPO_OBJETO)]
excel_bajada_objetos = excel_bajada_objetos[~pd.isnull(excel_bajada_objetos['CANTIDAD'])]

df_objetos = pd.DataFrame()
df_objetos['ID_PROCEDIMIENTO'] = excel_bajada_objetos['ID_PROCEDIMIENTO']
df_objetos['TIPO_INCAUTACION'] = "MERCADERIA"
df_objetos['TIPO'] = excel_bajada_objetos['TIPO_OBJETO']
df_objetos['SUBTIPO'] = "-"
df_objetos['CANTIDAD'] = excel_bajada_objetos['CANTIDAD']
df_objetos['MEDIDAS'] = "UNIDADES"
df_objetos['AFORO'] = "-"
df_objetos['OBSERVACIONES'] = "-"

df_objetos_completado = df_objetos[df_objetos['ID_PROCEDIMIENTO'].isin(df_procedimientos_final['ID_PROCEDIMIENTO'])]

print(df_objetos_completado.nunique())

Total de registros cargados: 680
ID_PROCEDIMIENTO    19
TIPO_INCAUTACION     1
TIPO                12
SUBTIPO              1
CANTIDAD             4
MEDIDAS              1
AFORO                1
OBSERVACIONES        1
dtype: int64


### INCAUTACIONES VEHICULOS

In [11]:
excel_bajada_vehiculos = pd.read_excel(PATH_VEHICULOS)
print(f"Total de registros vehiculos secuestrados: {len(excel_bajada_vehiculos)}")


excel_bajada_vehiculos = excel_bajada_vehiculos[excel_bajada_vehiculos["VEHICULO_ESTADO"] == "SECUESTRADO"]

excel_bajada_vehiculos['TIPO_CAUSA_INTERNA'] = excel_bajada_vehiculos.apply( procesar_tipo_causa_interna, axis=1)
excel_bajada_vehiculos['UOSP'] = excel_bajada_vehiculos['UOSP'].fillna(excel_bajada_vehiculos['URSA'])
excel_bajada_vehiculos['TIPO_CAUSA_INTERNA'] = excel_bajada_vehiculos.apply( procesar_tipo_causa_interna, axis=1)
excel_bajada_vehiculos['ID_PROCEDIMIENTO'] = excel_bajada_vehiculos.apply(generar_uid_sigpol, axis=1)


df_vehiculos = pd.DataFrame()
df_vehiculos['ID_PROCEDIMIENTO'] = excel_bajada_vehiculos['ID_PROCEDIMIENTO']
df_vehiculos['TIPO_INCAUTACION'] = "SECUESTRO DE VEHICULOS"
df_vehiculos['TIPO'] = excel_bajada_vehiculos['VEHICULO_TIPO']
df_vehiculos['SUBTIPO'] = "-"
df_vehiculos['CANTIDAD'] = excel_bajada_vehiculos['CANTIDAD']
df_vehiculos['MEDIDAS'] = "UNIDADES"
df_vehiculos['AFORO'] = "-"
df_vehiculos['OBSERVACIONES'] = excel_bajada_vehiculos.apply(observaciones_vehiculo,axis=1)


df_vehiculos_completado = df_vehiculos[df_vehiculos['ID_PROCEDIMIENTO'].isin(df_procedimientos_final['ID_PROCEDIMIENTO'])]

print(df_vehiculos_completado.nunique())

Total de registros vehiculos secuestrados: 37
ID_PROCEDIMIENTO    9
TIPO_INCAUTACION    1
TIPO                3
SUBTIPO             1
CANTIDAD            1
MEDIDAS             1
AFORO               1
OBSERVACIONES       8
dtype: int64


### INCAUTACIONES NARCOTRAFICO

In [12]:
# Paso 1: Cargar datos iniciales
excel_bajada_narcotrafico = pd.read_excel(PATH_NARCOTRAFICO)
print(f"Total de registros narcostrafico secuestrados: {len(excel_bajada_narcotrafico)}")

excel_bajada_narcotrafico['UOSP'] = excel_bajada_narcotrafico['UOSP'].fillna(excel_bajada_narcotrafico['URSA'])
excel_bajada_narcotrafico['TIPO_CAUSA_INTERNA'] = excel_bajada_narcotrafico.apply( procesar_tipo_causa_interna, axis=1)
excel_bajada_narcotrafico['ID_PROCEDIMIENTO'] = excel_bajada_narcotrafico.apply(generar_uid_sigpol, axis=1)

excel_bajada_narcotrafico = excel_bajada_narcotrafico[~pd.isnull(excel_bajada_narcotrafico['TIPO_ESTUPEFACIENTE'])]


#Paso 2: Crear df_divisa con las cantidades sumadas
df_narcotrafico = pd.DataFrame()
df_narcotrafico['ID_PROCEDIMIENTO'] = excel_bajada_narcotrafico['ID_PROCEDIMIENTO']
df_narcotrafico['TIPO_INCAUTACION'] = "ESTUPEFACIENTE"
df_narcotrafico['TIPO'] = excel_bajada_narcotrafico.apply(clasificar_tipo_sustancia, axis=1)
df_narcotrafico['SUBTIPO'] = "-"
df_narcotrafico[['CANTIDAD', 'MEDIDAS']] = excel_bajada_narcotrafico.apply(clasificar_medida, axis=1, result_type='expand')
df_narcotrafico['AFORO'] = "-"
df_narcotrafico['OBSERVACIONES'] = excel_bajada_narcotrafico.apply(observaciones_sustancia, axis=1)

df_narcotrafico_completado = df_narcotrafico[df_narcotrafico['ID_PROCEDIMIENTO'].isin(df_procedimientos_final['ID_PROCEDIMIENTO'])]

print(df_narcotrafico_completado.nunique())

Total de registros narcostrafico secuestrados: 67
ID_PROCEDIMIENTO    52
TIPO_INCAUTACION     1
TIPO                 5
SUBTIPO              1
CANTIDAD            33
MEDIDAS              3
AFORO                1
OBSERVACIONES        4
dtype: int64


### UNION DE INCAUTACIONES 
ARMAS, DIVISAS, OBJETOS , VEHICULOS , NARCOTRAFICO

In [13]:
df_incautados_completado= pd.concat([df_objetos_completado, df_vehiculos_completado, df_arma_completado, df_divisa_completado , df_narcotrafico_completado])
print(df_incautados_completado.nunique())

ID_PROCEDIMIENTO    76
TIPO_INCAUTACION     5
TIPO                29
SUBTIPO              1
CANTIDAD            50
MEDIDAS              3
AFORO                1
OBSERVACIONES       19
dtype: int64


### TRATA DE PERSONA

In [14]:
excel_bajada_trata = pd.read_excel(PATH_TRATA)
print(f"Total de registros tratas: {len(excel_bajada_trata)}")

excel_bajada_trata['UOSP'] = excel_bajada_trata['UOSP'].fillna(excel_bajada_trata['URSA'])
excel_bajada_trata['TIPO_CAUSA_INTERNA'] = excel_bajada_trata.apply( procesar_tipo_causa_interna, axis=1)
excel_bajada_trata['ID_PROCEDIMIENTO'] = excel_bajada_trata.apply(generar_uid_sigpol, axis=1)

# Crear DataFrame de detenidos y aprendidos
df_trata = pd.DataFrame()
df_trata['ID_PROCEDIMIENTO'] = excel_bajada_trata['ID_PROCEDIMIENTO']
df_trata['TIPO_DELITO'] = excel_bajada_trata["TIPO_EXPLOTACION"].fillna("-")
df_trata['SEXO_VICTIMA'] = excel_bajada_trata.apply(procesar_sexo, axis=1)
df_trata['GENERO_VICTIMA'] = excel_bajada_trata.apply(procesar_genero, axis=1)
df_trata['EDAD_VICTIMA'] = excel_bajada_trata['EDAD'].fillna("-")
df_trata['NACIONALIDAD'] = excel_bajada_trata['NACIONALIDAD'].fillna("-")
df_trata['NACIONALIDAD'] = excel_bajada_trata['NACIONALIDAD'].str.upper()
df_trata['JUZGADO_INTERVINIENTE'] = excel_bajada_trata.apply(procesar_juzgado, axis=1)
df_trata['OBSERVACIONES'] = "-"

df_trata_completado = df_trata[df_trata['ID_PROCEDIMIENTO'].isin(df_procedimientos_final['ID_PROCEDIMIENTO'])]


print(df_trata_completado.nunique())


Total de registros tratas: 2
ID_PROCEDIMIENTO         2
TIPO_DELITO              1
SEXO_VICTIMA             2
GENERO_VICTIMA           2
EDAD_VICTIMA             2
NACIONALIDAD             1
JUZGADO_INTERVINIENTE    2
OBSERVACIONES            1
dtype: int64


# OPERACIONES

### ORDEN DE SERVICIOS

In [15]:
excel_bajada_operaciones = pd.read_excel(PATH_OPERACIONES, 
                                         sheet_name="ORDEN_SERVICIOS", 
                                         skiprows=1)

# Usar .strftime('%H:%M') en cada valor para obtener solo la hora y el minuto
excel_bajada_operaciones['HORA'] = excel_bajada_operaciones['HORA'].apply(lambda x: x.strftime('%H:%M') if pd.notnull(x) else None)
# Ahora copiamos esta columna al nuevo DataFrame


df_operaciones= pd.DataFrame()
df_operaciones["ID_PROCEDIMIENTO"] = excel_bajada_operaciones["ID_PROCEDIMIENTO"]
df_operaciones["FUERZA_INTERVINIENTE"] = "PSA"
df_operaciones["ID_OPERATIVO"] = excel_bajada_operaciones["ID_OPERATIVO"]
df_operaciones["UNIDAD_INTERVINIENTE"] = excel_bajada_operaciones["UNIDAD_INTERVINIENTE"]
df_operaciones["DESCRIPCIÓN"] = excel_bajada_operaciones["DESCRIPCIÓN"]
df_operaciones["TIPO_INTERVENCION"] = excel_bajada_operaciones["TIPO_INTERVENCION"]
df_operaciones["PROVINCIA"] = excel_bajada_operaciones["PROVINCIA"].str.upper().replace("_", " ")
df_operaciones["DEPARTAMENTO O PARTIDO"] = excel_bajada_operaciones["DEPARTAMENTO O PARTIDO"].str.upper()
df_operaciones["LOCALIDAD"] = excel_bajada_operaciones["LOCALIDAD"]
df_operaciones["DIRECCION"] = excel_bajada_operaciones["DIRECCION"]
df_operaciones['FECHA'] = excel_bajada_operaciones["FECHA"]
df_operaciones['HORA'] = excel_bajada_operaciones['HORA']
df_operaciones["ZONA_SEGURIDAD_FRONTERAS"] = "-"
df_operaciones["PASO_FRONTERIZO"] = "-"
df_operaciones['OTRAS AGENCIAS INTERVINIENTES'] =  excel_bajada_operaciones["OTRAS AGENCIAS INTERVINIENTES"]
df_operaciones['Observaciones - Detalles'] = "PATRULLAJE DINAMICO"
df_operaciones[['LATITUD', 'LONGITUD']] = excel_bajada_operaciones.apply(procesar_geog_oper, axis=1, result_type='expand')


df_operaciones_final = df_operaciones[['FUERZA_INTERVINIENTE', 'ID_OPERATIVO', 'ID_PROCEDIMIENTO',
                                     'UNIDAD_INTERVINIENTE', 'DESCRIPCIÓN', 'TIPO_INTERVENCION',
                                     'PROVINCIA', 'DEPARTAMENTO O PARTIDO', 'LOCALIDAD', 'DIRECCION',
                                     'ZONA_SEGURIDAD_FRONTERAS', 'PASO_FRONTERIZO', 'LATITUD', 'LONGITUD',
                                     'FECHA', 'HORA', 'OTRAS AGENCIAS INTERVINIENTES', 'Observaciones - Detalles']]



# Reemplazar "S/D" y "N/C" por "-"
df_operaciones_final.replace(["S/D", "N/C"], "-", inplace=True)

# Reemplazar los valores vacíos (NaN) por "-"
df_operaciones_final.fillna("-", inplace=True)


print(df_operaciones_final.nunique())


FUERZA_INTERVINIENTE               1
ID_OPERATIVO                      95
ID_PROCEDIMIENTO                 277
UNIDAD_INTERVINIENTE              22
DESCRIPCIÓN                       81
TIPO_INTERVENCION                  2
PROVINCIA                         10
DEPARTAMENTO O PARTIDO            13
LOCALIDAD                         20
DIRECCION                         58
ZONA_SEGURIDAD_FRONTERAS           1
PASO_FRONTERIZO                    1
LATITUD                           54
LONGITUD                          60
FECHA                              9
HORA                              61
OTRAS AGENCIAS INTERVINIENTES      3
Observaciones - Detalles           1
dtype: int64


c:\Users\DICA\Desktop\Repositorio\DNEC\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


### HOJA DE VEHI. Y PERSO. CONTROLADAS

In [16]:
df_controlados_controlado = pd.DataFrame()
df_controlados_controlado["UID"] = excel_bajada_operaciones["ID_PROCEDIMIENTO"]
df_controlados_controlado["FUERZA_INTERVINIENTE"] = "PSA"
df_controlados_controlado["ID_PROCEDIMIENTO"] = excel_bajada_operaciones["ID_PROCEDIMIENTO"]
df_controlados_controlado["ID_OPERATIVO"] = excel_bajada_operaciones["ID_OPERATIVO"]
df_controlados_controlado["UNIDAD_INTERVINIENTE"] = excel_bajada_operaciones["UNIDAD_INTERVINIENTE"]
df_controlados_controlado["DESCRIPCIÓN"] = excel_bajada_operaciones["DESCRIPCIÓN"]
df_controlados_controlado["TIPO_INTERVENCION"] = excel_bajada_operaciones["TIPO_INTERVENCION"]
df_controlados_controlado["VEHICULOS_CONTROLADOS"] = excel_bajada_operaciones["VEHICULOS_CONTROLADOS"]
df_controlados_controlado["PERSONAS_CONTROLADAS"] = excel_bajada_operaciones["PERSONAS_CONTROLADAS"]
df_controlados_controlado["CANT_AVERIGUACIONES_SECUESTRO"] = excel_bajada_operaciones["CANT_AVERIGUACIONES_SECUESTRO"]
df_controlados_controlado["CANT_SOLICITUDES_ANTECEDENTES"] = excel_bajada_operaciones["CANT_SOLICITUDES_ANTECEDENTES"]


# Reemplazar "S/D" y "N/C" por "-"
df_controlados_controlado.replace(["S/D", "N/C"], "-", inplace=True)

# Reemplazar los valores vacíos (NaN) por "-"
df_controlados_controlado.fillna("-", inplace=True)

print(df_controlados_controlado.nunique())


UID                              277
FUERZA_INTERVINIENTE               1
ID_PROCEDIMIENTO                 277
ID_OPERATIVO                      95
UNIDAD_INTERVINIENTE              22
DESCRIPCIÓN                       81
TIPO_INTERVENCION                  2
VEHICULOS_CONTROLADOS             20
PERSONAS_CONTROLADAS              26
CANT_AVERIGUACIONES_SECUESTRO      1
CANT_SOLICITUDES_ANTECEDENTES     28
dtype: int64


C:\Users\DICA\AppData\Local\Temp\ipykernel_48224\1778806577.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_controlados_controlado.fillna("-", inplace=True)


### PERSONAL Y ELEMENTOS AFECTADOS

In [17]:
df_afectados_completado = pd.DataFrame()


df_afectados_completado["UID"] = excel_bajada_operaciones["ID_PROCEDIMIENTO"]
df_afectados_completado["FUERZA_INTERVINIENTE"] = "PSA"
df_afectados_completado["ID_PROCEDIMIENTO"] = excel_bajada_operaciones["ID_PROCEDIMIENTO"]
df_afectados_completado["ID_OPERATIVO"] = excel_bajada_operaciones["ID_OPERATIVO"]
df_afectados_completado["UNIDAD_INTERVINIENTE"] = excel_bajada_operaciones["UNIDAD_INTERVINIENTE"]
df_afectados_completado["DESCRIPCIÓN"] = excel_bajada_operaciones["DESCRIPCIÓN"]
df_afectados_completado["TIPO_INTERVENCION"] = excel_bajada_operaciones["TIPO_INTERVENCION"]
df_afectados_completado["CANT_EFECTIVOS"] = excel_bajada_operaciones["CANT_EFECTIVOS"]
df_afectados_completado["CANT_AUTOS_CAMIONETAS"] = excel_bajada_operaciones["CANT_AUTOS_CAMIONETAS"]
df_afectados_completado["CANT_SCANNERS"] = excel_bajada_operaciones["CANT_SCANNERS"]
df_afectados_completado["CANT_EMBARCACIONES"] = excel_bajada_operaciones["CANT_EMBARCACIONES"]
df_afectados_completado["CANT_MOTOS"] = excel_bajada_operaciones["CANT_MOTOS"]
df_afectados_completado["CANT_CABALLOS"] = excel_bajada_operaciones["CANT_CABALLOS"]
df_afectados_completado["CANT_CANES"] = excel_bajada_operaciones["CANT_CANES"]
df_afectados_completado["CANT_MORPHRAPID"] = excel_bajada_operaciones["CANT_MORPHRAPID"]
df_afectados_completado["CANT_LPR"] = excel_bajada_operaciones["CANT_LPR"]


# Reemplazar "S/D" y "N/C" por "-"
df_afectados_completado.replace(["S/D", "N/C"], "-", inplace=True)

# Reemplazar los valores vacíos (NaN) por "-"
df_afectados_completado.fillna("-", inplace=True)


print(df_afectados_completado.nunique())

UID                      277
FUERZA_INTERVINIENTE       1
ID_PROCEDIMIENTO         277
ID_OPERATIVO              95
UNIDAD_INTERVINIENTE      22
DESCRIPCIÓN               81
TIPO_INTERVENCION          2
CANT_EFECTIVOS            24
CANT_AUTOS_CAMIONETAS      7
CANT_SCANNERS              2
CANT_EMBARCACIONES         1
CANT_MOTOS                 1
CANT_CABALLOS              1
CANT_CANES                 3
CANT_MORPHRAPID            1
CANT_LPR                   1
dtype: int64


C:\Users\DICA\AppData\Local\Temp\ipykernel_48224\2143040399.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_afectados_completado.fillna("-", inplace=True)


### CODIGO OPERATIVO

In [18]:
df_codigo_operativo_final = pd.DataFrame()

df_codigo_operativo_final["UID"] = excel_bajada_operaciones["ID_PROCEDIMIENTO"]
df_codigo_operativo_final["FUERZA_INTERVINIENTE"] = "PSA"
df_codigo_operativo_final["ID_PROCEDIMIENTO"] = excel_bajada_operaciones["ID_PROCEDIMIENTO"]
df_codigo_operativo_final["ID_OPERATIVO"] = excel_bajada_operaciones["ID_OPERATIVO"]
df_codigo_operativo_final["UNIDAD_INTERVINIENTE"] = excel_bajada_operaciones["UNIDAD_INTERVINIENTE"]
df_codigo_operativo_final["DESCRIPCIÓN"] = excel_bajada_operaciones["DESCRIPCIÓN"]
df_codigo_operativo_final["TIPO_INTERVENCION"] = excel_bajada_operaciones["TIPO_INTERVENCION"]
df_codigo_operativo_final["CODIGO_OPERATIVO"] = excel_bajada_operaciones["CODIGO_OPERATIVO"]


# Reemplazar "S/D" y "N/C" por "-"
df_codigo_operativo_final.replace(["S/D", "N/C"], "-", inplace=True)

# Reemplazar los valores vacíos (NaN) por "-"
df_codigo_operativo_final.fillna("-", inplace=True)


print(df_codigo_operativo_final.nunique())

UID                     277
FUERZA_INTERVINIENTE      1
ID_PROCEDIMIENTO        277
ID_OPERATIVO             95
UNIDAD_INTERVINIENTE     22
DESCRIPCIÓN              81
TIPO_INTERVENCION         2
CODIGO_OPERATIVO          4
dtype: int64


C:\Users\DICA\AppData\Local\Temp\ipykernel_48224\1120024258.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_codigo_operativo_final.fillna("-", inplace=True)


# UNION DE DE DATAFRAMES 

 CONCATENACION DE PROCEDIMIENTOS Y ORDEN DE SERVICIOS , TAMBIEN SE UNEN LAS 

In [19]:

df_geog_final = pd.concat ([df_procedimientos_final, df_operaciones_final]) 

print(df_geog_final.nunique())

FUERZA_INTERVINIENTE               1
ID_OPERATIVO                     407
ID_PROCEDIMIENTO                 604
UNIDAD_INTERVINIENTE              42
DESCRIPCIÓN                       89
TIPO_INTERVENCION                  2
PROVINCIA                         25
DEPARTAMENTO O PARTIDO            30
LOCALIDAD                         20
DIRECCION                         78
ZONA_SEGURIDAD_FRONTERAS           1
PASO_FRONTERIZO                    1
LATITUD                          132
LONGITUD                         141
FECHA                             35
HORA                             205
OTRAS AGENCIAS INTERVINIENTES      3
Observaciones - Detalles           2
dtype: int64


COMPLETO LA INFORMACION DE TRATA Y OTROS DELITOS DADO QUE "TENGO" QUE PEGAR TODA LA INFORMACION DE LA GEOG FINAL

In [20]:
# *TIENE QUE TRAER LA INFORMACION DE GEOG FINAL Y COMPLETAR LA INFORMACION 
df_trata_final = pd.merge(df_geog_final, df_trata_completado, on='ID_PROCEDIMIENTO', how='right')
print("OTROS TRATA FINAL")

df_trata_final = df_trata_final[[
    'FUERZA_INTERVINIENTE',
    'ID_OPERATIVO',
    'ID_PROCEDIMIENTO',
    'UNIDAD_INTERVINIENTE',
    'DESCRIPCIÓN',
    'TIPO_INTERVENCION',
    "TIPO_DELITO",
    'SEXO_VICTIMA',
    'GENERO_VICTIMA',
    'EDAD_VICTIMA',
    'NACIONALIDAD',
    'JUZGADO_INTERVINIENTE',
    'OBSERVACIONES'
]]
print(df_trata_final.nunique())

df_otros_delitos_final = pd.merge(df_geog_final, df_otros_delitos_completado, on='ID_PROCEDIMIENTO', how='right')
print("\nOTROS DELITOS FINAL")

df_otros_delitos_final = df_otros_delitos_final[[
    'FUERZA_INTERVINIENTE',
    'ID_OPERATIVO',
    'ID_PROCEDIMIENTO',
    'UNIDAD_INTERVINIENTE',
    'DESCRIPCIÓN',
    'TIPO_INTERVENCION',
    "TIPO_OTRO_DELITO",
    'GENERO_VICTIMA',
    'EDAD_VICTIMA',
    'NACIONALIDAD',
    'OBSERVACIONES'
]]

print(df_otros_delitos_final.nunique())

OTROS TRATA FINAL
FUERZA_INTERVINIENTE     1
ID_OPERATIVO             2
ID_PROCEDIMIENTO         2
UNIDAD_INTERVINIENTE     2
DESCRIPCIÓN              2
TIPO_INTERVENCION        2
TIPO_DELITO              1
SEXO_VICTIMA             2
GENERO_VICTIMA           2
EDAD_VICTIMA             2
NACIONALIDAD             1
JUZGADO_INTERVINIENTE    2
OBSERVACIONES            1
dtype: int64

OTROS DELITOS FINAL
FUERZA_INTERVINIENTE      1
ID_OPERATIVO            137
ID_PROCEDIMIENTO        137
UNIDAD_INTERVINIENTE     19
DESCRIPCIÓN               5
TIPO_INTERVENCION         1
TIPO_OTRO_DELITO         16
GENERO_VICTIMA            2
EDAD_VICTIMA             55
NACIONALIDAD             12
OBSERVACIONES             1
dtype: int64


COMPLETO LA INFORMACION DE TRATA Y OTROS DELITOS DADO QUE NO TENGO QUE PEGAR TODA LA INFORMACION DE LA GEOG FINAL

In [21]:

# *TIENE QUE PEGAR TODAS LAS FILAS DE  GEOG AUNQUE NO TENGA INFORMACION ES DECIR TIENE QUE SER MAYOR A LA HOJA GEO

df_incautados_final= pd.merge(df_geog_final, df_incautados_completado, on='ID_PROCEDIMIENTO', how='left')
df_incautados_final = df_incautados_final[[
    'FUERZA_INTERVINIENTE',
    'ID_OPERATIVO',
    'ID_PROCEDIMIENTO',
    'UNIDAD_INTERVINIENTE',
    'DESCRIPCIÓN',
    'TIPO_INTERVENCION',
    'TIPO_INCAUTACION',
    'TIPO',
    'SUBTIPO',
    'CANTIDAD',
    'MEDIDAS',
    "AFORO",
    'OBSERVACIONES'
]]


print("\nINCAUTADOS FINAL")
print(df_incautados_final.nunique())

df_detenidos_aprendidos_final = pd.merge(df_geog_final, df_detenidos_aprendidos_completado, on='ID_PROCEDIMIENTO', how='left')

print("\nDETENIDOS FINAL")
df_detenidos_aprendidos_final = df_detenidos_aprendidos_final[[
    'FUERZA_INTERVINIENTE',
    'ID_OPERATIVO',
    'ID_PROCEDIMIENTO',
    'UNIDAD_INTERVINIENTE',
    'DESCRIPCIÓN',
    'TIPO_INTERVENCION',
    'EDAD',
    'SEXO',
    'GENERO',
    'NACIONALIDAD',
    'SITUACION_PROCESAL',
    "DELITO_IMPUTADO",
    'JUZGADO_INTERVINIENTE',
    'CARATULA_CAUSA',
    "NUM_CAUSA"
]]
print(df_detenidos_aprendidos_final.nunique())




print("\nAFECTADOS FINAL")
df_afectados_final = pd.merge(df_geog_final, df_afectados_completado, on='ID_PROCEDIMIENTO', how='left',     suffixes=('', '_afectados')  # Evitar sufijos por defecto
 )
df_afectados_final = df_afectados_final[[
    'FUERZA_INTERVINIENTE',
    'ID_OPERATIVO',
    'ID_PROCEDIMIENTO',
    'UNIDAD_INTERVINIENTE',
    'DESCRIPCIÓN',
    'TIPO_INTERVENCION',
    'CANT_EFECTIVOS',
    'CANT_AUTOS_CAMIONETAS',
    'CANT_SCANNERS',
    'CANT_EMBARCACIONES',
    'CANT_MOTOS',
    "CANT_CABALLOS",
    'CANT_CANES',
    'CANT_MORPHRAPID',
    "CANT_LPR"
]]

print(df_afectados_final.nunique())





df_controlados_final = pd.merge(df_geog_final, df_controlados_controlado, on='ID_PROCEDIMIENTO', how='left', suffixes=('', '_controlados'))
print("\nCONTROLADOS FINAL")

df_controlados_final = df_controlados_final[[
    'FUERZA_INTERVINIENTE',
    'ID_OPERATIVO',
    'ID_PROCEDIMIENTO',
    'UNIDAD_INTERVINIENTE',
    'DESCRIPCIÓN',
    'TIPO_INTERVENCION',
    'VEHICULOS_CONTROLADOS',
    'PERSONAS_CONTROLADAS',
    "CANT_AVERIGUACIONES_SECUESTRO",
    'CANT_SOLICITUDES_ANTECEDENTES',
]]

print(df_controlados_final.nunique())




df_codigo_operativo_final = pd.merge(df_geog_final, df_codigo_operativo_final, on='ID_PROCEDIMIENTO', how='left',suffixes=('', '_codigos'))
print("\nCODIGO FINAL")


df_codigo_operativo_final = df_codigo_operativo_final[[
    'FUERZA_INTERVINIENTE',
    'ID_OPERATIVO',
    'ID_PROCEDIMIENTO',
    'UNIDAD_INTERVINIENTE',
    'DESCRIPCIÓN',
    'TIPO_INTERVENCION',
    'CODIGO_OPERATIVO',
]]
print(df_codigo_operativo_final.nunique())




INCAUTADOS FINAL
FUERZA_INTERVINIENTE      1
ID_OPERATIVO            407
ID_PROCEDIMIENTO        604
UNIDAD_INTERVINIENTE     42
DESCRIPCIÓN              89
TIPO_INTERVENCION         2
TIPO_INCAUTACION          5
TIPO                     29
SUBTIPO                   1
CANTIDAD                 50
MEDIDAS                   3
AFORO                     1
OBSERVACIONES            19
dtype: int64

DETENIDOS FINAL
FUERZA_INTERVINIENTE       1
ID_OPERATIVO             407
ID_PROCEDIMIENTO         604
UNIDAD_INTERVINIENTE      42
DESCRIPCIÓN               89
TIPO_INTERVENCION          2
EDAD                      50
SEXO                       2
GENERO                     2
NACIONALIDAD              15
SITUACION_PROCESAL         5
DELITO_IMPUTADO           15
JUZGADO_INTERVINIENTE     64
CARATULA_CAUSA            55
NUM_CAUSA                157
dtype: int64

AFECTADOS FINAL
FUERZA_INTERVINIENTE       1
ID_OPERATIVO             407
ID_PROCEDIMIENTO         604
UNIDAD_INTERVINIENTE      42
DESCRIP

# GENERACIÓN DE INFORME

 ### CREACION DE INFORME SEMANAL CON FECHA HOY

In [22]:

dataframes = {
    'GEOG. PROCEDIMIENTO': df_geog_final.reset_index(drop=True).fillna("").replace("", "-"),
    'VEHI. Y PERSO. CONTROLADAS': df_controlados_final.reset_index(drop=True).fillna("").replace("", "-"),
    'PERSONAL Y ELEMENTOS AFECTADOS': df_afectados_final.reset_index(drop=True).fillna("").replace("", "-"),
    'INCAUTACIONES': df_incautados_final.reset_index(drop=True).fillna("").replace("", "-"),
    'DETENIDOS Y APREHENDIDOS': df_detenidos_aprendidos_final.reset_index(drop=True).fillna("").replace("", "-"),
    'OTROS DELITOS': df_otros_delitos_final.reset_index(drop=True).fillna("").replace("", "-"),
    'TRATA O TRAFIC PERSONAS': df_trata_final.reset_index(drop=True).fillna("").replace("", "-"),
    'CODIGO OPERATIVO': df_codigo_operativo_final.reset_index(drop=True).fillna("").replace("", "-")
}


# Cargar la plantilla
wb = load_workbook('Planilla_modelo.xlsx')

# Escribir los datos
for sheet_name, df in dataframes.items():
    sheet = wb[sheet_name]
    for i, row in df.iterrows():
        for j, value in enumerate(row):
            cell= sheet.cell(row=i+4, column=j+2, value=value) # type: ignore
            cell.alignment = Alignment(horizontal='center')
            

for sheet in wb.worksheets:
    sheet.conditional_formatting = []
    sheet.auto_filter.ref = None
    sheet.auto_filter = None
# Guardar el archivo
output_file = f'Informe_semanal_{datetime.now().strftime("%Y%m%d")}.xlsx'
wb.save(output_file)
print(f"Archivo generado: {output_file}")





Archivo generado: Informe_semanal_20241210.xlsx


### GUARDAR EN LA BASE DE DATOS LA INFORMACION ACTUALIZADA

In [23]:


# Verificar si el archivo de Excel existe
if os.path.exists(PATH_BASE):
    # Cargar el archivo de Excel existente
    book_base = load_workbook(PATH_BASE)
else:
    # Crear un nuevo archivo de Excel
    book_base = Workbook()

# Iterar sobre cada hoja del archivo
for sheet_name in wb.sheetnames:
    sheet = wb[sheet_name]
    
    # Obtener el último índice de fila con datos
    last_row = sheet.max_row
    
    # Concatenar la información al final de la hoja
    for df_name, df in dataframes.items():
        if df_name == sheet_name:
            for index, row in df.iterrows():
                sheet.append(list(row))
    
    # Guardar los cambios
    wb.save(PATH_BASE)
    
    
print(f"Base de datos actualizada: {PATH_BASE}")


Base de datos actualizada: db/base_informada.xlsx
